# 36610 - Python and Unix for Bioinformaticians

## Project 6: Resistance to antibiotics

Details on how to read compressed files can be found [here](https://docs.python.org/3/library/gzip.html#gzip.decompress).

In [12]:
import gzip # import gzip to decompress .gz files

resistance_genes = open("data/resistance_genes.fsa", "r")
reads1 = gzip.open("data/Unknown3_raw_reads_1.txt.gz", "rt") # 'rt' -> read-in in text mode
reads2 = gzip.open("data/Unknown3_raw_reads_2.txt.gz", "rt")

In [1]:
import time         # time measurement
start = time.time() # time measurement

k_mer_length = 19

read_kmer_set = set()        # set will contain k-mers of sequencing reads
all_sequencing_reads = set() # stores sequencing reads in set

def read_in_seq_reads(read, kmer_length):
    """Extract sequencing read sequences and generate k-mers from them. Store in separate sets."""
    raw_reads = open("data/Unknown3_raw_" + read + ".txt", "r")
    
    # Variables
    read_seq = ""
    read_seq_rev_comp = ""
    line_count = 3
    
    for read_line in raw_reads:
        read_line = read_line
        if (line_count % 4 == 0):                                # get sequence line
            read_seq = read_line.rstrip()
            read_seq_rev_comp = reverse_complement(read_seq)     # consider other strand (reverse complement)
        if (read_seq is not "" or read_seq_rev_comp is not ""):
            # store read k-mer and its complement taken from middle portion of seq in a set.
            read_kmer_set.add(read_seq[44:44+kmer_length])
            read_kmer_set.add(read_seq_rev_comp[44:44+kmer_length])
            # store sequencing reads in a separate set ->loop only once through sequencing read file
            all_sequencing_reads.add(read_seq)
            all_sequencing_reads.add(read_seq_rev_comp)
        line_count += 1
    raw_reads.close()

def reverse_complement(DNA_sequence):
    """Reverse complement a given DNA sequence"""
    DNA_translation_table = str.maketrans("ACGT", "TGCA")
    rev_compliment = DNA_sequence.translate(DNA_translation_table)
    return rev_compliment[::-1]

def k_mer_set_of_RES(sequence, kmer_length):
    '''Return a set of k-mers of given sequence.
    Used to search the single read k-mer in this set'''
    k_mer_set = set()
    for i in range(0, len(sequence), 1):
        if (i < len(sequence)-kmer_length+1):
            k_mer_set.add(sequence[i:i+kmer_length])
    return(k_mer_set)

def find_them_all(read_seq, res_seq):
    """Find all positions of a read in a resistance gene"""
    i = res_seq.find(read_seq)
    while (i != -1):
        yield i
        i = res_seq.find(read_seq, i + 1) # increase counter to prevent duplicates

def determine_coverage_depth(full_seq, kmer_intersection):
    # initialize a list with length of resistance gene and set values to 0
    sequence_depth = [0 for i in range(len(full_seq))]

    #print("Be aware that does not mean the entire read aligns to the sequence. Can still result in 0's.")

    # For all sequencing reads: if its k-mer is in the intersection (thus matches a resistance gene kmer),
    # get all position to which the full read can potentially align in the resistance gene.
    # We will obtain the starting position of the alignment. Based on that we can increase
    # the coverage depth for the region that is covered by the entire read.
    for read_seq in all_sequencing_reads:
        if (read_seq[44:44+k_mer_length] in kmer_intersection):
            hit_positions = find_them_all(read_seq, full_seq) # read may align to multiple positions
            for hit in hit_positions:
                for i in range(hit, hit+len(read_seq), 1):
                    sequence_depth[i] += 1
    return sequence_depth

def depth_coverage_filter(depth_list, len_of_seq):
    """95% sequence coverage, at least depth of 10"""
    count_of_zeros = 0
    sum_of_depth = 0
    
    # reject sequence if single depth value is below 10
    for i in depth_list:
    #    if (i < 10):
    #        return False
        sum_of_depth += i
    
    # this would be a way to calculate the coverage, but it's not necessary because a depth<10 already rejects the sequence?!
    # hence we would never count bases with coverage == 0
        if (i == 0):
            count_of_zeros += 1
    
    # coverage (>95%)
    coverage = (len_of_seq - count_of_zeros) / len_of_seq
    #if (coverage < 0.95):
    #    return False
    
    # depth (>10)
    avg_depth = sum_of_depth / len_of_seq
    #if (avg_depth < 10):
    #    return False
    
    return(coverage, avg_depth)

print("Reading reads...")

read_in_seq_reads("reads_1", k_mer_length)
read_in_seq_reads("reads_2", k_mer_length)

print("Set contains in total {} read k-mers.".format(len(read_kmer_set)))
print("Number of sequencing reads including reverse complements is: {}".format(len(all_sequencing_reads)))
print("Done setting up sets.")
print("Determining coverage and depth...")

# Variables
fasta = []            # will temporarily store the sequence for each resistance gene
final_result = list() # stores final output

ResFile = open("data/resistance_genes.fsa", "r")
for line in ResFile:
    line = line.rstrip()
    
    # Arrived at resistance gene header. Collect its sequence.
    # Once we get to the next resistance gene entry, process the previous one.
    if line.startswith(">"):
        
        # Once we have the complete resistance gene sequence, generate k-mers of it. 
        # Find those k-mers that are identical to the read k-mers (using set intersection)
        if fasta:
            full_seq = "".join(fasta)
            res_kmer_set = k_mer_set_of_RES(full_seq, k_mer_length)
            kmer_intersection = res_kmer_set.intersection(read_kmer_set)

            # Following 'if' can probably be more stringent i.e. allow only a minimum number of k-mers
            # that are in common, but I suspect it would NOT drastically speed up the program.
            if (len(kmer_intersection) > 10):
                
                ## Idea - but too slow: get those reads whose k-mer was in the intersection
                #reads_of_interest = [ele for ele in all_sequencing_reads for x in kmer_intersection if x in ele]
                
                sequence_depth = determine_coverage_depth(full_seq, kmer_intersection)
                                
                # Once we processed all the reads, we can apply the filter.
                # 1) 95% coverage of the resistance gene
                # 2) Coverage depth of at least 10
                coverage, avg_depth = depth_coverage_filter(sequence_depth, len(sequence_depth))
                
                # If it passes the filter, store it in a separate list for final evaluation
                if (coverage > 0.95 and avg_depth > 10):
                    final_result.append([header, coverage, avg_depth])
            fasta = []
        header = line
        #print(header)
    else:
        sequence = line
        fasta.append(sequence)

# process last FASTA entry
if fasta:
    full_seq = "".join(fasta)
    res_kmer_set = k_mer_set_of_RES(full_seq, k_mer_length)
    kmer_intersection = res_kmer_set.intersection(read_kmer_set)
    if (len(kmer_intersection) > 10):
        sequence_depth = determine_coverage_depth(full_seq, kmer_intersection)
        coverage, avg_depth = depth_coverage_filter(sequence_depth, len(sequence_depth))
        if (coverage > 0.95 and avg_depth > 10):
            final_result.append([header, coverage, avg_depth])

ResFile.close()

# Sort by
# 1) Coverage, then
# 2) Depth
final_result.sort(key=lambda x: (x[1], x[2]))
print("Cov\t\tAvg depth\tGene")
for i in range(len(final_result)):
    print("{:.2f}\t\t{:.2f}\t\t{}".format(final_result[i][1]*100, final_result[i][2], final_result[i][0]))

end = time.time()
elapsed = end - start
elapsed

Reading reads...
Set contains in total 8201883 read k-mers.
Number of sequencing reads including reverse complements is: 11022032
Done setting up sets.
Determining coverage and depth...
Cov		Avg depth	Gene
95.01		19.36		>blaTEM-97_1_AF397067 Beta-lactam resistance:Alternate name; blaTEM-34
95.10		15.22		>sul2_6_FN995456 Sulphonamide resistance:
95.19		19.20		>sul2_9_FJ197818 Sulphonamide resistance:
95.31		29.41		>blaOXA-320_1_KF151169 Beta-lactam resistance:
95.38		19.07		>aac(6')Ib-cr_2_EF636461 Fluoroquinolone and aminoglycoside resistance:
95.43		22.02		>blaCTX-M-22_1_HM470254 Beta-lactam resistance:
95.47		26.98		>blaTEM-104_1_AF516719 Beta-lactam resistance:
95.70		23.70		>blaTEM-99_1_AF397066 Beta-lactam resistance:Alternate name; blaTEM-30
95.89		23.06		>blaCTX-M-28_6_AJ549244 Beta-lactam resistance:
95.93		12.90		>blaTEM-93_1_AJ318093 Beta-lactam resistance:
96.05		15.84		>blaTEM-183_1_HQ529916 Beta-lactam resistance:
96.05		23.34		>blaTEM-55_1_DQ286729 Beta-lactam resistance:

2291.5162189006805

## New approach

1. Generate a **resistance gene dictionary of kmers** (RES_GENE_DICT), which looks like this:
<pre><code class="python">{gene_header1:{kmer1_1:0, kmer1_2:0, kmer1_3:0}, gene_header2:{kmer2_1:0, kmer2_2:0, kmer2_3:0}, ...}</code></pre>

2. Loop over **read files**: generate read kmers and check if in RES_GENE_DICT. If so, increase corresponding value by 1. **BUT:** This results in too many comparisons/checks because we loop over the read files (which contains 3.5M reads each) and generate kmers of it which are then compared to the kmers of the 2000 resistance genes in the dict.

3. Determine **coverage** and **depth**, and print genes that pass the filter (>95% coverage, >10 depth). Sorted output.

In [ ]:
import time         # time measurement
start = time.time() # time measurement

ResFile = open("data/resistance_genes.fsa", "r")

fasta = [] # will contain the full resistance gene sequence
kmer_length = 19
RES_kmer_dict = {} # dictionary {header:{kmer1:count1, kmer2:count2, ...}}

# !!!
# If RES kmers are the same, they are represented by a single kmer in the dictionary? Problem?
# !!!

# Variables for read file
read_seq = ""
read_seq_rev_comp = ""
line_count = 3

for line in ResFile:
    line = line.rstrip()
    
    # Arrived at resistance gene header. Collect its sequence.
    # Once we get to the next resistance gene entry, process the previous one.
    if line.startswith(">"):
        
        # Once we have the complete resistance gene sequence, generate k-mers of it.
        # feed it into a dictionary
        if fasta:
            full_seq = "".join(fasta)
            
            RES_kmer_dict[header] = {}
            
            for i in range(0, len(full_seq), 1):
                if (i < len(full_seq) - kmer_length + 1):
                    RES_kmer_dict[header][full_seq[i:i+kmer_length]] = 0
            
            fasta = []
        header = line
    else:
        sequence = line
        fasta.append(sequence)
        
# process last resistance gene
full_seq = "".join(fasta)
RES_kmer_dict[header] = {}
            
for i in range(0, len(full_seq), 1):
    if (i < len(full_seq) - kmer_length + 1):
        RES_kmer_dict[header][full_seq[i:i+kmer_length]] = 0

ResFile.close()

print("Finished setting up RES dictionary.")
print("Processing reads...")

# Go through the read files
reads1 = open("data/Unknown3_raw_reads_1.txt", "r")

for read_line in reads1:
    if (line_count % 4 == 0):
        read_seq = read_line.rstrip()
        
        for j in range(0, len(read_seq), 1):
            if (j < len(read_seq) - kmer_length + 1):
                read_kmer = read_seq[j:j+kmer_length]
                
                # dictionary of read kmers
                
                # CPU intensive task:
                # for EACH kmer, go through the RES dict
                #
                # Alternative:
                # Store read kmer in a set (= does not contain duplicate kmers), and then loop through RES dictionary?? but then you have to loop over it again which takes time...
                #
                for kmer_combo in RES_kmer_dict.values():
                    if read_kmer in kmer_combo.keys():
                        kmer_combo[read_kmer] += 1
    line_count += 1

print("Done processing reads.")
print(RES_kmer_dict)
print("Filtering results...")

final_result = list()

for i, j in RES_kmer_dict.items():
    min_depth = 0
    number_of_res_kmers = len(j)
    
    kmer_above_zero = len([v for k, v in j.items() if v > 0])
    coverage = int(kmer_above_zero * 100 / number_of_res_kmers)
    #print("Coverage: {}%".format(coverage))
    
    if coverage > 95:
        #avg_depth = sum([v for k, v in j.items()]) / number_of_res_kmers
        min_depth = min([v for k, v in j.items()])
        #print("Average depth: {}".format(avg_depth))
        if min_depth > 10:
            final_result.append([i, coverage, avg_depth])

final_result.sort(key=lambda x: (x[1], x[2]))
print("Cov\t\tAvg depth\tGene")
for i in range(len(final_result)):
    print("{:.2f}\t\t{:.2f}\t\t{}".format(final_result[i][1], final_result[i][2], final_result[i][0]))

reads1.close()

end = time.time()
elapsed = end - start
elapsed

Finished setting up RES dictionary.
Processing reads...


## May 4, 2019
### Peter's discussion
#### Set of RES kmer's to quickly check if sequencing read needs further investigation

In [7]:
## import gzip
import time         # time measurement
start = time.time() # time measurement

def reverse_complement(DNA_sequence):
    """Reverse complement a given DNA sequence"""
    DNA_translation_table = str.maketrans("ACGT", "TGCA")
    rev_compliment = DNA_sequence.translate(DNA_translation_table)
    return rev_compliment[::-1]

fasta = [] # will contain the full resistance gene sequence
kmer_length = 19
store_fasta_seqs = []

print("Generating RES set.")
ResKmerSet  = set()
ResKmerDict = dict()
ResFile = open("data/resistance_genes.fsa", "r")
for line in ResFile:
    line = line.rstrip()
    if line.startswith(">"):
        if fasta:
            full_seq = "".join(fasta)
            store_fasta_seqs.append(header)
            store_fasta_seqs.append(full_seq)
            for i in range(0, len(full_seq), 1):
                if (i < len(full_seq) - kmer_length + 1):
                    ResKmerSet.add(full_seq[i:i+kmer_length])
                    ResKmerDict[full_seq[i:i+kmer_length]] = 1
            fasta = []
        header = line
    else:
        sequence = line
        fasta.append(sequence)

# ToDo: process last resistance gene

ResFile.close()

print("Finished setting up RES set.")
print("Reading read file...")

line_count = 3
reads1 = gzip.open("data/Unknown3_raw_reads_1.txt.gz", "rt")
for read_line in reads1:
    if (line_count % 4 == 0):
        # include reverse complement check and process
        read_seq = read_line.rstrip()
        read_kmer_set = set()
        
        read_kmer_set.add(read_seq[1:1+kmer_length])
        read_kmer_set.add(read_seq[41:41+kmer_length])
        read_kmer_set.add(read_seq[81:81+kmer_length])
        
        if len(read_kmer_set.intersection(ResKmerSet)) >= 2:
            for j in range(0, len(read_seq), 1):
                if (j < len(read_seq) - kmer_length + 1):
                    read_kmer = read_seq[j:j+kmer_length]
                    if read_kmer in ResKmerDict.keys():
                        ResKmerDict[read_kmer] += 1
    line_count += 1
    
print("Determining whether sequence fulfills requirements...")

counter = 0
sequence = ""
for i in range(len(store_fasta_seqs)):
    if (counter % 2 == 0):
        header = store_fasta_seqs[i]
    else:
        sequence = store_fasta_seqs[i]
        for j in range(0, len(sequence), 1):
            if (j < len(sequence) - kmer_length + 1):
                read_kmer = sequence[j:j+kmer_length]
                 += ResKmerDict[read_kmer]
    counter += 1

print("Finished.")
end = time.time()
elapsed = end - start
elapsed

#for i, j in RES_kmer_dict.items():
#    min_depth = 0
#    number_of_res_kmers = len(j)
#    
#    kmer_above_zero = len([v for k, v in j.items() if v > 0])
#    coverage = int(kmer_above_zero * 100 / number_of_res_kmers)
#    #print("Coverage: {}%".format(coverage))
#    
#    if coverage > 95:
#        #avg_depth = sum([v for k, v in j.items()]) / number_of_res_kmers
#        min_depth = min([v for k, v in j.items()])
#        #print("Average depth: {}".format(avg_depth))
#        if min_depth > 10:
#            final_result.append([i, coverage, avg_depth])
#
#final_result.sort(key=lambda x: (x[1], x[2]))
#print("Cov\t\tAvg depth\tGene")
#for i in range(len(final_result)):
#    print("{:.2f}\t\t{:.2f}\t\t{}".format(final_result[i][1], final_result[i][2], final_result[i][0]))

Generating RES set.
Finished setting up RES set.
Reading read file...
Determining whether sequence fulfills requirements...
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322
324
326
328
330
332
334
336
338
340
342
344
346
348
350
352
354
356
358
360
362
364
366
368
370
372
374
376
378
380
382
384
386
388
390
392
394
396
398
400
402
404
406
408
410
412
414
416
418
420
422
424
426
428
430
432
434
436
438
440
442
444
446
448
450
452
454
456
458
460
462
464

20.959698915481567

- res genes can look alike/similar
- he uses .find()
- check only part of the read, if suffienctly enough kmers match, then go through each kmer of that read
- do NOT loop over the res gene dict (-> slow)

### Load reads in memory too

In [1]:
import time         # time measurement
start = time.time() # time measurement

def reverse_complement(DNA_sequence):
    """Reverse complement a given DNA sequence"""
    DNA_translation_table = str.maketrans("ACGT", "TGCA")
    rev_compliment = DNA_sequence.translate(DNA_translation_table)
    return rev_compliment[::-1]

fasta = [] # will contain the full resistance gene sequence
kmer_length = 19
RES_dict_from_read_kmers = {} # dictionary {header:{kmer1:count1, kmer2:count2, ...}}

# Variables for read file
read_seq = ""
read_seq_rev_comp = ""
line_count = 3

print("Processing reads...")

# Go through the read files
read_kmer_dict = {}

reads1 = open("data/Unknown3_raw_reads_1.txt", "r")

for read_line in reads1:
    if (line_count % 4 == 0):
        read_seq = read_line.rstrip()
        
        for j in range(0, len(read_seq), 1):
            if (j < len(read_seq) - kmer_length + 1):
                read_kmer = read_seq[j:j+kmer_length]
                rev_read_kmer = reverse_complement(read_kmer)
                
                if read_kmer in read_kmer_dict.keys():
                    read_kmer_dict[read_kmer] += 1
                else:
                    read_kmer_dict[read_kmer] = 1
                
                if rev_read_kmer in read_kmer_dict.keys():
                    read_kmer_dict[rev_read_kmer] += 1
                else:
                    read_kmer_dict[rev_read_kmer] = 1
    line_count += 1
    
line_count = 3
reads2 = open("data/Unknown3_raw_reads_2.txt", "r")
for read_line in reads2:
    if (line_count % 4 == 0):
        read_seq = read_line.rstrip()        
        for j in range(0, len(read_seq), 1):
            if (j < len(read_seq) - kmer_length + 1):
                read_kmer = read_seq[j:j+kmer_length]
                rev_read_kmer = reverse_complement(read_kmer)
                
                if read_kmer in read_kmer_dict.keys():
                    read_kmer_dict[read_kmer] += 1
                else:
                    read_kmer_dict[read_kmer] = 1
                
                if rev_read_kmer in read_kmer_dict.keys():
                    read_kmer_dict[rev_read_kmer] += 1
                else:
                    read_kmer_dict[rev_read_kmer] = 1
    line_count += 1

reads1.close()
reads2.close()
print("Done processing reads.")

ResFile = open("data/resistance_genes.fsa", "r")

for line in ResFile:
    line = line.rstrip()
    
    # Arrived at resistance gene header. Collect its sequence.
    # Once we get to the next resistance gene entry, process the previous one.
    if line.startswith(">"):
        
        # Once we have the complete resistance gene sequence, generate k-mers of it.
        # feed it into a dictionary
        if fasta:
            full_seq = "".join(fasta)
            
            RES_dict_from_read_kmers[header] = {}
            
            for i in range(0, len(full_seq), 1):
                if (i < len(full_seq) - kmer_length + 1):
                    RES_kmer = full_seq[i:i+kmer_length]
                    if RES_kmer in read_kmer_dict:
                        RES_dict_from_read_kmers[header][RES_kmer] = read_kmer_dict[RES_kmer]
                    else:
                        RES_dict_from_read_kmers[header][RES_kmer] = 0
            
            fasta = []
        header = line
    else:
        sequence = line
        fasta.append(sequence)
        
# ToDo: Process last entry
        
ResFile.close()

print("Finished setting up RES dictionary.")

#print(RES_dict_from_read_kmers)

print("Filtering results...")

final_result = list()

for i, j in RES_dict_from_read_kmers.items():
    min_depth = 0
    number_of_res_kmers = len(j)
    
    kmer_above_zero = len([v for k, v in j.items() if v > 0])
    coverage = int(kmer_above_zero * 100 / number_of_res_kmers)
    
    if coverage > 95:
        min_depth = min([v for k, v in j.items()])
        if min_depth > 10:
            final_result.append([i, coverage, min_depth])

final_result.sort(key=lambda x: (x[1], x[2]))
print("Cov\t\tMin depth\tGene")
for i in range(len(final_result)):
    print("{:.2f}\t\t{:.2f}\t\t{}".format(final_result[i][1], final_result[i][2], final_result[i][0]))

end = time.time()
elapsed = end - start
elapsed

Processing reads...
Done processing reads.
Finished setting up RES dictionary.
Filtering results...
Cov		Min depth	Gene
100.00		16.00		>sul2_2_GQ421466 Sulphonamide resistance:
100.00		19.00		>aac(3)-IIa_1_CP023555.1 Aminoglycoside resistance:
100.00		21.00		>blaCTX-M-15_23_DQ302097 Beta-lactam resistance:Alternate name; UOE-1
100.00		26.00		>blaTEM-1B_1_JF910132 Beta-lactam resistance:Alternate name; RblaTEM-1
100.00		27.00		>strB_1_M96392 Aminoglycoside resistance:Alternate name; aph(6)-Id
100.00		27.00		>blaOXA-1_1_J02967 Beta-lactam resistance:
100.00		29.00		>aac(6')Ib-cr_1_DQ303918 Fluoroquinolone and aminoglycoside resistance:
100.00		29.00		>strA_4_AF321551 Aminoglycoside resistance:Alternate name; aph(3'')-Ib
100.00		29.00		>catB4_1_EU935739 Phenicol resistance:
100.00		29.00		>tet(A)_4_AJ517790 Tetracycline resistance:
100.00		37.00		>blaSHV-28_1_HM751101 Beta-lactam resistance:


1321.9995760917664

### Test area

In [5]:
test_dict = {"head1":{"key1_1":2, "key1_2":0, "key1_3":4}, "head2":{"key2_1":20, "key2_2":22}}
print(test_dict.items())
# test_dict["head1"]["key1"] += 1

# test 1
if "key1" in test_dict:
    print("Yes")
else:
    print("Nope")

# test 2
for k in test_dict.values():
    print(k)

# test 3
for j in test_dict.values():
    if "key1_1" in j.keys():
        j["key1_1"] += 1

print(test_dict)
print("\n")

final_result = list()

for i,j in test_dict.items():
    print(j)
    number_of_res_kmers = len(j)
    #print(number_of_res_kmers)
    
    avg_depth = sum([v for k, v in j.items()]) / number_of_res_kmers
    print("Average depth: {}".format(avg_depth))
    

    kmer_above_zero = len([v for k, v in j.items() if v > 0])
    coverage = int(kmer_above_zero * 100 / number_of_res_kmers)
    print("Coverage: {}%".format(coverage))

    if avg_depth > 10 or coverage > 95:
        final_result.append([i, coverage, avg_depth])

final_result.sort(key=lambda x: (x[1], x[2]))
print("Cov\t\tAvg depth\tGene")
for i in range(len(final_result)):
    print("{:.2f}\t\t{:.2f}\t\t{}".format(final_result[i][1], final_result[i][2], final_result[i][0]))

dict_items([('head1', {'key1_1': 2, 'key1_2': 0, 'key1_3': 4}), ('head2', {'key2_1': 20, 'key2_2': 22})])
Nope
{'key1_1': 2, 'key1_2': 0, 'key1_3': 4}
{'key2_1': 20, 'key2_2': 22}
YES IT IS
{'head1': {'key1_1': 3, 'key1_2': 0, 'key1_3': 4}, 'head2': {'key2_1': 20, 'key2_2': 22}}


{'key1_1': 3, 'key1_2': 0, 'key1_3': 4}
Average depth: 2.3333333333333335
Coverage: 66%
{'key2_1': 20, 'key2_2': 22}
Average depth: 21.0
Coverage: 100%
Cov		Avg depth	Gene
100.00		21.00		head2
